In [4]:
import pandas as pd

MASTER_FILE = r"C:\Users\Siddu\Downloads\New folder\ds-rpc-02\data\processed\master_vaers_data.csv"
SAMPLE_FILE = r"C:\Users\Siddu\Downloads\New folder\ds-rpc-02\data\processed\gold_data_for_annotation.csv"
SAMPLE_SIZE = 500

In [5]:
df = pd.read_csv(MASTER_FILE,encoding="latin1")
df.head()

,VAERS_ID,SYMPTOM_TEXT,AGE_YRS,SEX,structured_symptoms_list,vax_names,vax_manus
0,810053,Extreme shivering for approx. 5 hours then fev...,73.0,F,"['Asthenia', 'Pain', 'Chills', 'Pyrexia', 'Hea...",['INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE)'],['SANOFI PASTEUR']
1,855017,Symptoms occurred almost immediately- aching j...,55.0,F,"['Arthralgia', 'Chills', 'Injection site pain'...",['ZOSTER (SHINGRIX)'],['GLAXOSMITHKLINE BIOLOGICALS']
2,855018,"Extreme pain, muscle weakness in right arm so ...",68.0,F,"['Chills', 'Muscular weakness', 'Fatigue', 'Pa...",['VACCINE NOT SPECIFIED (OTHER)'],['UNKNOWN MANUFACTURER']
3,855019,SORENESS IN THE AREA. ITCHING AND RASH,50.0,F,"['Pain', 'Pruritus', 'Rash']",['ZOSTER (SHINGRIX)'],['GLAXOSMITHKLINE BIOLOGICALS']
4,855020,"sore arm, fever 101 , chills, and aching musc...",67.0,F,"['Chills', 'Influenza like illness', 'Myalgia'...",['ZOSTER (SHINGRIX)'],['GLAXOSMITHKLINE BIOLOGICALS']


In [6]:
df.isnull().sum()

VAERS_ID                         0
SYMPTOM_TEXT                     0
AGE_YRS                     164868
SEX                           1933
structured_symptoms_list         0
vax_names                        0
vax_manus                        0
dtype: int64

In [7]:
sample_df = df.sample(n=SAMPLE_SIZE, random_state=42)

In [8]:
output_df = sample_df[['VAERS_ID', 'SYMPTOM_TEXT']]
output_df.head()

,VAERS_ID,SYMPTOM_TEXT
1120971,2632957,J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/1...
1243448,2838275,"Site: Pain at Injection Site-Mild, Additional ..."
286339,1171003,Extreme fatigue x 2 days On day 4 started havi...
1177964,2728589,Pt was given MMRV (Proquad) and Varicella vacc...
247801,1126654,"chills, fever, tiredness, headache, sore muscl..."


In [6]:
output_df.to_csv(SAMPLE_FILE, index=False)

In [9]:
output_df['SYMPTOM_TEXT']

1120971    J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/1...
1243448    Site: Pain at Injection Site-Mild, Additional ...
286339     Extreme fatigue x 2 days On day 4 started havi...
1177964    Pt was given MMRV (Proquad) and Varicella vacc...
247801     chills, fever, tiredness, headache, sore muscl...
                                 ...                        
187567     Large itchy rash from top of arm at injection ...
128486     The day following vaccination had terrible hea...
741043             Right Armpit and breast pain and swelling
971864     Covid-19 vaccine was administered at 0845 on 0...
485586     Stomach shocks in left abdomen started about 3...
Name: SYMPTOM_TEXT, Length: 500, dtype: object

In [17]:
import re
import json
import ast # For safely evaluating list-like strings

MASTER_FILE = r"C:\Users\Siddu\Downloads\New folder\ds-rpc-02\data\processed\master_vaers_data.csv"
OUTPUT_FILE = r"C:\Users\Siddu\Downloads\New folder\ds-rpc-02\data\processed\weak_data.json"
SAMPLE_SIZE = 100000

GENERIC_DRUG_KEYWORDS = [
    "vaccine", "vaccination", "booster shot",
    "COVID-19 vaccinated"
]


In [13]:
def safe_parse_list(list_str):
    """
    Example: "['Headache', 'Fever']" -> ['Headache', 'Fever']
    """
    try:
        parsed_list = ast.literal_eval(list_str)
        if isinstance(parsed_list, list):
            return [str(item) for item in parsed_list]
        return []
    except (ValueError, SyntaxError, TypeError):
        return []

In [15]:
def create_weak_labels(row):

    text = row['SYMPTOM_TEXT']
    if not isinstance(text, str):
        return None 
        
    text_lower = text.lower()
    spans = [] 

    ade_keywords = set(safe_parse_list(row['structured_symptoms_list']))
    
    vax_names = safe_parse_list(row['vax_names'])
    vax_manus = safe_parse_list(row['vax_manus'])

    drug_keywords = set(vax_names + vax_manus + GENERIC_DRUG_KEYWORDS)

    def find_matches(keywords, label):
        found_spans = []
        for keyword in keywords:
            if not keyword or not isinstance(keyword, str):
                continue
            try:
                for match in re.finditer(r'\b' + re.escape(keyword.lower()) + r'\b', text_lower):
                    start, end = match.span()
                    found_spans.append({"from_name": "label",
                                        "to_name": "text",
                                        "type": "labels",
                                        "value": {
                                            "start": start,
                                            "end": end,
                                            "text": text[start:end],
                                            "labels": [label]
                                        }})
            except re.error:
                continue
        return found_spans
    
    ade_spans = find_matches(ade_keywords, "ADE")
    drug_spans = find_matches(drug_keywords, "DRUG")


    if not ade_spans and not drug_spans:
        return None
        
    label_studio_format = {
        "data": {
            "text": text
        },
        "annotations": [
            {
                "result": ade_spans + drug_spans
            }
        ]
    }
    return label_studio_format

In [18]:
print(f"Loading master file: {MASTER_FILE}")
df = pd.read_csv(MASTER_FILE)
df = df.dropna(subset=['SYMPTOM_TEXT'])

Loading master file: C:\Users\Siddu\Downloads\New folder\ds-rpc-02\data\processed\master_vaers_data.csv


In [19]:
if SAMPLE_SIZE and SAMPLE_SIZE < len(df):
    print(f"Sub-sampling {SAMPLE_SIZE} random rows for weak labeling...")
    df_sample = df.sample(n=SAMPLE_SIZE, random_state=42)
else:
    print(f"Using all {len(df)} rows for weak labeling...")
    df_sample = df

Sub-sampling 100000 random rows for weak labeling...


In [21]:
weak_labels_list = df_sample.apply(create_weak_labels, axis=1)

final_labels = [item for item in weak_labels_list if item is not None]

In [22]:
final_labels

[{'data': {'text': 'J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 CHF (CONGESTIVE HEART FAILURE), UNSPECIFIED J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 CHF (CONGESTIVE HEART FAILURE), UNSPECIFIED J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 CKD STAGE 3B (GFR 30-44) J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 UNSPECIFIED ANXIETY DISORDER J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 DIASTOLIC HEART FAILURE, ACUTE ON CHRONIC J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 CHF (CONGESTIVE HEART FAILURE), UNSPECIFIED J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/15/2022 CHF EXACERBATION, UNSPECIFIED'},
  'annotations': [{'result': [{'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'value': {'start': 337,
       'end': 353,
       'text': 'ANXIETY DISORDER',
       'labels': ['ADE']}}]}]},
 {'data': {'text': 'Site: Pain at Injection Site-Mild, Additional Details: Patient reported that he experiences pain at injection 

In [23]:
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(final_labels, f, indent=2)

In [24]:
df_sample.head()

,VAERS_ID,SYMPTOM_TEXT,AGE_YRS,SEX,structured_symptoms_list,vax_names,vax_manus
1120971,2632957,J96.01 ACUTE HYPOXEMIC RESPIRATORY FAILURE 5/1...,54.0,M,"['Acute respiratory failure', 'Condition aggra...",['COVID19 (COVID19 (PFIZER-BIONTECH))'],['PFIZER\\BIONTECH']
1243448,2838275,"Site: Pain at Injection Site-Mild, Additional ...",67.0,M,['Injection site pain'],['COVID19 (COVID19 (MODERNA))'],['MODERNA']
286339,1171003,Extreme fatigue x 2 days On day 4 started havi...,39.0,F,"['Chills', 'Myalgia', 'Fatigue', 'Nausea', 'He...",['COVID19 (COVID19 (MODERNA))'],['MODERNA']
1177964,2728589,Pt was given MMRV (Proquad) and Varicella vacc...,1.0,M,"['Extra dose administered', 'No adverse event'...",['INFLUENZA (SEASONAL) (FLUZONE QUADRIVALENT)'...,"['SANOFI PASTEUR', 'MERCK & CO. INC.', 'MERCK ..."
247801,1126654,"chills, fever, tiredness, headache, sore muscl...",74.0,M,"['Chills', 'Tenderness', 'Headache', 'Injectio...",['COVID19 (COVID19 (MODERNA))'],['MODERNA']


In [25]:
OUTPUT_FILE

'C:\\Users\\Siddu\\Downloads\\New folder\\ds-rpc-02\\data\\processed\\weak_data.json'